## 1. Importar Librerías

In [ ]:
import openai
import os
from dotenv import load_dotenv

## 2. Cargar Configuración

In [ ]:
load_dotenv()

print(f"Modelo: {os.getenv('MODEL')}")

## 3. Configurar el Cliente de OpenAI

In [ ]:
openai_client = openai.AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_APIKEY"),
    api_version=os.getenv("AZURE_OPENAI_VERSION"),
)

## 4. Definir Parámetros de la Conversación

Definimos:
- El tema a debatir
- Las personalidades de cada asistente
- Número de iteraciones (turnos de conversación)

In [ ]:
tema_a_tratar = "¿Quien es mejor equipo: el Real Madrid o el FC Barcelona?"
personalidad_asistente_1 = "Soy del Real Madrid y tengo muy mala leche. Soy maleducado y borde. No soporto al FC Barcelona."
personalidad_asistente_2 = "Soy del FC Barcelona y me gusta mucho pinchar a los del Real Madrid. Soy sarcástico y bromista. Muchas veces soy insoportable."
iteraciones = 5  # Número de turnos de conversación

print("=" * 80)
print("CONFIGURACIÓN DE LA CONVERSACIÓN")
print("=" * 80)
print(f"\nTema: {tema_a_tratar}")
print(f"\nAsistente 1 (Real Madrid): {personalidad_asistente_1}")
print(f"\nAsistente 2 (FC Barcelona): {personalidad_asistente_2}")
print(f"\nIteraciones: {iteraciones}")
print("\n" + "=" * 80)

## 5. Inicializar Historiales de Conversación

Cada asistente tiene su propio historial de mensajes. Ambos comienzan con un mensaje de sistema que define su personalidad.

In [ ]:
messages_1 = [
    {"role": "system", "content": personalidad_asistente_1},
]

messages_2 = [
    {"role": "system", "content": personalidad_asistente_2},
]

print("✅ Historiales de conversación inicializados")

## 6. Iniciar la Conversación

El Asistente 2 (Barcelona) recibe el tema como pregunta inicial.

In [ ]:
question = tema_a_tratar
messages_2.append({"role": "user", "content": question})

model = os.getenv("MODEL")

print("\n" + "=" * 80)
print("INICIO DE LA CONVERSACIÓN")
print("=" * 80 + "\n")
print(f"Tema inicial: {question}\n")

## 7. Loop de Conversación

En cada iteración:
1. Asistente 2 responde
2. Su respuesta se guarda y se pasa al Asistente 1
3. Asistente 1 responde
4. Su respuesta se guarda y se pasa de vuelta al Asistente 2

Esto crea un ciclo de debate continuo.

In [ ]:
for i in range(iteraciones):
    print(f"\n{'─' * 80}")
    print(f"TURNO {i+1}")
    print(f"{'─' * 80}\n")
    
    # Asistente 2 (Barcelona) responde
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages_2
    )
    question = response.choices[0].message.content
    messages_2.append({"role": "assistant", "content": question})
    messages_1.append({"role": "user", "content": question})
    print(f"🔵 Asistente 2 (Barcelona): {question}")

    # Asistente 1 (Madrid) responde
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages_1
    )
    question = response.choices[0].message.content
    messages_1.append({"role": "assistant", "content": question})
    messages_2.append({"role": "user", "content": question})
    print(f"\n⚪ Asistente 1 (Madrid): {question}")

print("\n" + "=" * 80)
print("FIN DE LA CONVERSACIÓN")
print("=" * 80)

## 8. Estadísticas de la Conversación

Veamos cuántos mensajes se intercambiaron.

In [ ]:
print("\n📊 ESTADÍSTICAS")
print(f"Total de mensajes Asistente 1: {len(messages_1)}")
print(f"Total de mensajes Asistente 2: {len(messages_2)}")
print(f"Total de intercambios: {iteraciones * 2}")

## 9. Ver Historial Completo (Opcional)

Si quieres revisar todo el historial de uno de los asistentes:

In [ ]:
print("\n📜 HISTORIAL COMPLETO - ASISTENTE 1 (MADRID)\n")
for i, msg in enumerate(messages_1[1:], 1):  # Omitimos el system prompt
    role = "🤖 Asistente" if msg['role'] == 'assistant' else "👤 Usuario"
    print(f"{i}. {role}: {msg['content'][:100]}...\n")

## Conclusión

Este ejemplo demuestra:

### 1. **Conversación Multi-Agente**
- Dos instancias independientes del modelo
- Cada una mantiene su propio contexto y personalidad
- Los mensajes fluyen de uno a otro

### 2. **Gestión de Estado**
- Historiales de conversación separados
- Cada asistente ve el historial desde su perspectiva
- Consistencia de personalidad a lo largo de la conversación

### 3. **System Prompts Efectivos**
- Definición clara de personalidad y comportamiento
- Los modelos mantienen su rol consistentemente
- Genera conversaciones naturales y entretenidas

### Aplicaciones Prácticas:
- Simulaciones de debates
- Entrenamiento de modelos de diálogo
- Generación de contenido conversacional
- Testing de diferentes estrategias de conversación
- Juegos de rol y entretenimiento

### Advertencia:
Este es un ejemplo recreativo. En producción, considera:
- Límites de iteraciones para evitar bucles infinitos
- Moderación de contenido
- Costos de API (cada iteración hace 2 llamadas)
- Timeouts y manejo de errores